In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from pandas_datareader.data import DataReader
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from cvxopt import matrix, solvers

In [4]:
#download data of ETFs
ticker = ["FXE","EWJ","GLD","QQQ","SPY","SHV","DBA","USO",
          "XBI","ILF","GAF","EPP","FEZ"]
start =  datetime(2007, 1, 1)
end = datetime(2016, 10, 20)
data = pd.DataFrame()
for i in ticker:
    data[i] = DataReader(i, 'yahoo', start, end)["Close"]
data.to_csv("ETFs.csv")

In [5]:
#load data
ETF = pd.read_csv("ETFs.csv", index_col=0)[55:]
F = pd.read_csv("Factors.csv", index_col=0)[56:]
F.index = ETF.index[1:]
#calculte the simples anualized returns for the ETFs
R = (ETF.pct_change(1)[1:])*250
#calculate the excess annualized return for the ETFs
ER = pd.DataFrame(R.values-F["RF"].values.reshape(-1,1),
                  index=F.index, columns=ticker)
F = F.iloc[:,0:3]

In [59]:
# Before crisis("2007-03-26"-"2008-03-23")
R_bc = R["2007-03-26":"2008-03-23"].values
ER_bc = ER["2007-03-26":"2008-03-23"].values
F_bc = F["2007-03-26":"2008-03-23"].values

#short term model(60 days)
Lambda = 0.001
beta_T = [0.5, 1, 1.5]
R_opt = []

#conduct the max return strategy
for j in beta_T:
    Rp = []
    wp = np.ones((13,1))*1/13
    for i in range(len(R_bc)-59):
        r = R_bc[i:(i+60),:]
        er = ER_bc[i:(i+60),:]
        f1 = F_bc[i:(i+60),:]
        #rho = r.mean(axis=0).reshape(-1,1)
        cov_f = np.cov(f1, rowvar=False)
        #run regression to get the beta
        lm = LinearRegression()
        lm.fit(f1, er)
        coeff3 = lm.coef_
        beta = coeff3[:,0]
        error = er - lm.predict(f1)
        #calculate the covariance matrix
        Q = coeff3.dot(cov_f).dot(coeff3.T)+np.diag(error.var(axis=0))
        #preparation for the optimization
        P = matrix(2*Lambda*Q, tc='d')
        q = matrix(-2*Lambda*(Q.T).dot(wp)-rho, tc='d')
        A = matrix(np.vstack((beta, [1]*13)), tc='d')
        G = matrix(np.vstack((np.diag([1]*13),np.diag([-1]*13))), tc='d')
        h = matrix([2]*26, tc='d')
        b = matrix([j,1], tc='d')
        #do the optimization using QP solver
        opt = solvers.qp(P, q, G, h, A, b, options={'show_progress':False})
        w = opt['x']
        wp = np.array(w).reshape(-1,1)
        Rp = Rp + [wp.T.dot(rho)[0,0]]
    R_opt.append(Rp)

#conduct the min variance with 15% target return strategy.
Rp = []
wp = np.ones((13,1))*1/13
for i in range(len(R_bc)-59):
    r = R_bc[i:(i+60),:]
    er = ER_bc[i:(i+60),:]
    f2 = F_bc[i:(i+60),:]
    #rho = r.mean(axis=0)
    cov_f = np.cov(f2, rowvar=False)
    #run regression to get the beta
    lm = LinearRegression()
    lm.fit(f2, er)
    coeff3 = lm.coef_
    beta = coeff3[:,0]
    error = er - lm.predict(f2)
    #calculate the covariance matrix
    Q = coeff3.dot(cov_f).dot(coeff3.T)+np.diag(error.var(axis=0))
    #preparation for the optimization
    P = matrix(2*(1+Lambda)*Q, tc='d')
    q = matrix(-2*Lambda*(Q.T).dot(wp), tc='d')
    G = matrix(np.vstack((np.diag([1]*13),np.diag([-1]*13))), tc='d')
    h = matrix([2]*26, tc='d') 
    A = matrix(np.vstack((rho, [1]*13)), tc='d')
    b = matrix([0.15,1], tc='d')
    #do the optimization using QP solver
    opt = solvers.qp(P, q, G, h, A, b, options={'show_progress':False})
    w = opt['x']
    wp = np.array(w).reshape(-1,1)
    Rp = Rp + [wp.T.dot(rho.reshape(-1,1))[0,0]]
R_opt.append(Rp)

In [60]:
pd.DataFrame(R_opt,index=['beta=0.5','beta=1','beta=1.5','minvar']).T

,beta=0.5,beta=1,beta=1.5,minvar
0,2.257344,2.325261,2.393177,0.15
1,2.330352,2.387678,2.445003,0.15
2,2.476637,2.536215,2.595793,0.15
3,2.446423,2.501371,2.556319,0.15
4,2.489088,2.558504,2.627917,0.15
5,3.023889,3.094497,3.154537,0.15
6,2.790643,2.835047,2.879452,0.15
7,2.459691,2.501026,2.542360,0.15
8,2.613355,2.648594,2.683833,0.15
9,2.507273,2.548906,2.590539,0.15


In [57]:
r = R_bc[0:60,:]
er = ER_bc[0:60,:]
f1 = F_bc[0:60,:]
rho = r.mean(axis=0).reshape(-1,1)
cov_f = np.cov(f1, rowvar=False)
#run regression to get the beta
lm = LinearRegression()
lm.fit(f1, er)
coeff3 = lm.coef_
beta = coeff3[:,0]
error = er - lm.predict(f1)
#calculate the covariance matrix
Q = coeff3.dot(cov_f).dot(coeff3.T)+np.diag(error.var(axis=0))
#preparation for the optimization
P = matrix(2*Lambda*Q, tc='d')
q = matrix(-2*Lambda*(Q.T).dot(wp)-rho, tc='d')
A = matrix(np.vstack((beta, [1]*13)), tc='d')
b = matrix([1.5,1], tc='d')
#do the optimization using QP solver
opt = solvers.qp(P=P, q=q, A=A, b=b, options={'show_progress':False})
w = opt['x']
wp = np.array(w).reshape(-1,1)

In [58]:
wp.T.dot(rho)

array([[ 125.59504919]])

In [25]:
wp

array([[ -25.5979569 ],
       [-180.60668393],
       [ -31.29573352],
       [  53.11018336],
       [-134.71171865],
       [ 199.27409129],
       [   5.5852627 ],
       [  10.07203765],
       [  25.36192474],
       [  77.65961165],
       [ -23.38346073],
       [  48.90665289],
       [ -23.37421055]])

0
1
0
1
2
